In [1]:
state = 'IA' 
year = 2020
obj_type='cut_edges'

In [2]:
filepath = 'C:\\districting-data-2020-reprojection\\'
filename = state + '_county.json'

from read import read_graph_from_json
G = read_graph_from_json( state, filepath + filename) 
k = G._k

In [3]:
G._ideal_population = sum( G.nodes[i]['TOTPOP'] for i in G.nodes ) / k
print(f"When k = {k}, ideal_population = {round(G._ideal_population,2)}.")

When k = 4, ideal_population = 797592.25.


In [4]:
from metrics import dominates,scores
def filter_dominated_plans_old(G, plans, obj_type):
    nondominated_plans = list()
    senses = ['max' if obj_type in ['bottleneck_Polsby_Popper', 'average_Polsby_Popper'] else 'min', 'min']
    
    for plan1 in plans:
        if not any(dominates(scores(G, plan2, G._ideal_population, obj_type), 
                            scores(G, plan1, G._ideal_population, obj_type), senses) 
                    for plan2 in plans):
            nondominated_plans.append(plan1)
    
    return nondominated_plans

In [5]:
def filter_dominated_plans_new(G, plans, obj_type):
    nondominated_plans = []
    compactness_sense = 'max' if obj_type in ['bottleneck_Polsby_Popper', 'average_Polsby_Popper'] else 'min'
    scored_plans = [(scores(G, plan, G._ideal_population, obj_type), plan) for plan in plans]

   
    scored_plans.sort(key=lambda x: (
        x[0][0],                                 # deviation
        -x[0][1] if compactness_sense == 'max'   # negate for descending sort
        else x[0][1]                             # ascending
    ))

    best_compactness = None

    for (deviation, compactness), plan in scored_plans:
        if best_compactness is None or ((compactness > best_compactness) if compactness_sense == 'max' else (compactness < best_compactness)):
            best_compactness = compactness
            nondominated_plans.append(plan)

    return nondominated_plans


In [6]:
from IA_plans_2020 import plans
print(f"Loaded {len(plans)} plans from file.")
warm_starts = plans

Loaded 5000 plans from file.


In [7]:
plans = [[[1, 2, 4, 8, 21, 22, 27, 41, 48, 53, 59, 60, 68, 71, 76, 78, 79, 86, 92], [0, 5, 9, 14, 17, 19, 20, 23, 30, 31, 32, 33, 35, 36, 37, 40, 42, 43, 49, 52, 55, 57, 58, 62, 63, 64, 65, 66, 70, 73, 74, 81, 84, 89, 91, 93], [7, 11, 13, 15, 25, 34, 47, 56, 61, 69, 72, 77, 80, 82, 87, 88, 90], [3, 6, 10, 12, 16, 18, 24, 26, 28, 29, 38, 39, 44, 45, 46, 50, 51, 54, 67, 75, 83, 85, 94, 95, 96, 97, 98]], [[4, 8, 17, 21, 22, 46, 48, 60, 68, 71, 76, 78, 86, 94, 96], [0, 1, 2, 9, 14, 19, 20, 23, 27, 30, 31, 32, 33, 35, 37, 40, 41, 42, 43, 49, 50, 52, 53, 55, 57, 58, 59, 62, 63, 64, 65, 66, 70, 73, 79, 81, 84, 89, 91, 92, 93], [11, 15, 25, 61, 72, 82, 87, 88, 90, 98], [3, 5, 6, 7, 10, 12, 13, 16, 18, 24, 26, 28, 29, 34, 36, 38, 39, 44, 45, 47, 51, 54, 56, 67, 69, 74, 75, 77, 80, 83, 85, 95, 97]], [[4, 8, 16, 17, 48, 49, 60, 68, 71, 76, 78, 79, 86, 94], [0, 1, 2, 9, 14, 19, 20, 21, 22, 23, 27, 30, 31, 32, 33, 35, 37, 40, 41, 42, 43, 50, 52, 53, 55, 57, 58, 59, 62, 63, 64, 65, 66, 70, 73, 81, 84, 89, 91, 92, 93], [12, 24, 25, 26, 39, 44, 47, 61, 72, 88, 96, 97, 98], [3, 5, 6, 7, 10, 11, 13, 15, 18, 28, 29, 34, 36, 38, 45, 46, 51, 54, 56, 67, 69, 74, 75, 77, 80, 82, 83, 85, 87, 90, 95]], [[1, 2, 8, 14, 20, 21, 22, 27, 41, 59, 60, 65, 68, 71, 76, 78, 79, 84, 92], [0, 4, 9, 16, 17, 19, 23, 30, 31, 32, 33, 35, 36, 37, 40, 42, 43, 49, 50, 52, 53, 55, 57, 58, 62, 63, 64, 66, 70, 73, 74, 81, 89, 91, 93], [7, 11, 15, 25, 34, 47, 51, 56, 61, 72, 77, 80, 82, 87, 88, 90], [3, 5, 6, 10, 12, 13, 18, 24, 26, 28, 29, 38, 39, 44, 45, 46, 48, 54, 67, 69, 75, 83, 85, 86, 94, 95, 96, 97, 98]], [[0, 4, 8, 17, 49, 78, 93], [1, 2, 9, 12, 14, 19, 20, 21, 22, 23, 27, 30, 31, 32, 33, 35, 37, 40, 41, 42, 43, 46, 48, 52, 53, 55, 57, 58, 59, 60, 62, 63, 64, 65, 66, 68, 70, 71, 73, 76, 79, 81, 84, 86, 89, 91, 92, 94, 96], [11, 15, 24, 25, 26, 39, 44, 47, 61, 67, 72, 82, 87, 88, 97], [3, 5, 6, 7, 10, 13, 16, 18, 28, 29, 34, 36, 38, 45, 50, 51, 54, 56, 69, 74, 75, 77, 80, 83, 85, 90, 95, 98]], [[0, 4, 8, 17, 46, 49, 81, 93, 94], [1, 2, 9, 14, 19, 20, 21, 22, 23, 27, 30, 31, 32, 33, 35, 37, 40, 41, 42, 43, 48, 52, 53, 55, 57, 58, 59, 60, 62, 63, 64, 65, 66, 68, 70, 71, 73, 74, 76, 78, 79, 84, 86, 89, 91, 92, 96], [5, 6, 7, 11, 13, 16, 28, 29, 34, 36, 45, 50, 51, 56, 69, 75, 77, 80, 82, 85, 88, 90], [3, 10, 12, 15, 18, 24, 25, 26, 38, 39, 44, 47, 54, 61, 67, 72, 83, 87, 95, 97, 98]], [[4, 8, 17, 18, 48, 66, 76, 78, 94], [0, 1, 2, 9, 14, 19, 20, 21, 22, 23, 27, 30, 31, 32, 33, 35, 37, 40, 41, 42, 43, 49, 52, 53, 55, 58, 59, 60, 62, 63, 64, 65, 68, 70, 71, 73, 79, 81, 84, 86, 89, 91, 92, 93], [7, 11, 15, 25, 34, 47, 51, 56, 61, 72, 77, 80, 82, 87, 88, 90], [3, 5, 6, 10, 12, 13, 16, 24, 26, 28, 29, 36, 38, 39, 44, 45, 46, 50, 54, 57, 67, 69, 74, 75, 83, 85, 95, 96, 97, 98]], [[4, 8, 48, 60, 68, 71, 76, 78, 86, 93], [0, 1, 2, 9, 14, 17, 19, 20, 21, 22, 23, 27, 30, 31, 32, 33, 35, 36, 37, 40, 41, 42, 43, 49, 50, 52, 53, 55, 57, 58, 59, 62, 63, 64, 65, 66, 70, 73, 74, 79, 81, 84, 89, 91, 92], [12, 24, 25, 26, 39, 44, 47, 61, 72, 88, 96, 97, 98], [3, 5, 6, 7, 10, 11, 13, 15, 16, 18, 28, 29, 34, 38, 45, 46, 51, 54, 56, 67, 69, 75, 77, 80, 82, 83, 85, 87, 90, 94, 95]], [[8, 17, 18, 37, 48, 53, 68, 76, 78, 93, 94], [0, 1, 2, 4, 9, 14, 19, 20, 21, 22, 23, 27, 30, 31, 32, 33, 35, 40, 41, 42, 43, 49, 52, 55, 57, 58, 59, 60, 62, 63, 64, 65, 66, 70, 71, 73, 79, 84, 86, 89, 91, 92], [5, 6, 7, 10, 11, 13, 16, 28, 29, 34, 36, 45, 50, 51, 54, 56, 69, 74, 75, 77, 80, 81, 83, 85, 88, 90], [3, 12, 15, 24, 25, 26, 38, 39, 44, 46, 47, 61, 67, 72, 82, 87, 95, 96, 97, 98]], [[2, 4, 8, 16, 17, 48, 60, 68, 71, 76, 78, 86, 94, 96], [0, 1, 9, 14, 19, 20, 21, 22, 23, 27, 30, 31, 32, 33, 35, 37, 40, 41, 42, 43, 49, 52, 53, 55, 57, 58, 59, 62, 63, 64, 65, 66, 70, 73, 79, 81, 84, 89, 91, 92, 93], [11, 15, 25, 61, 72, 82, 87, 88, 90, 98], [3, 5, 6, 7, 10, 12, 13, 18, 24, 26, 28, 29, 34, 36, 38, 39, 44, 45, 46, 47, 50, 51, 54, 56, 67, 69, 74, 75, 77, 80, 83, 85, 95, 97]]]*2000

In [8]:
print(len(plans))

20000


In [9]:
plans = plans+warm_starts
print(len(plans))

25000


In [10]:
import time
start_time = time.perf_counter()
nondominated_plans_old = filter_dominated_plans_old(G, plans, obj_type)
print("Total time for filtering dominated plans =", round(time.perf_counter() - start_time, 2))

Total time for filtering dominated plans = 54579.21


In [14]:
print(nondominated_plans_old)

[[[1, 2, 8, 14, 20, 21, 22, 27, 41, 59, 60, 65, 68, 71, 76, 78, 79, 84, 92], [5, 6, 7, 11, 13, 15, 29, 34, 45, 51, 56, 69, 72, 75, 77, 80, 82, 85, 87, 88, 90], [3, 10, 12, 18, 24, 25, 26, 28, 38, 39, 44, 46, 47, 48, 54, 61, 67, 83, 86, 94, 95, 96, 97, 98], [0, 4, 9, 16, 17, 19, 23, 30, 31, 32, 33, 35, 36, 37, 40, 42, 43, 49, 50, 52, 53, 55, 57, 58, 62, 63, 64, 66, 70, 73, 74, 81, 89, 91, 93]], [[1, 2, 8, 20, 21, 22, 27, 41, 59, 60, 65, 68, 71, 76, 78, 79, 84, 92], [5, 6, 7, 11, 13, 15, 29, 34, 36, 45, 51, 56, 69, 74, 75, 77, 80, 82, 85, 87, 88, 90], [3, 12, 18, 24, 25, 26, 28, 38, 39, 44, 46, 47, 48, 54, 61, 67, 72, 83, 86, 94, 95, 96, 97, 98], [0, 4, 9, 10, 14, 16, 17, 19, 23, 30, 31, 32, 33, 35, 37, 40, 42, 43, 49, 50, 52, 53, 55, 57, 58, 62, 63, 64, 66, 70, 73, 81, 89, 91, 93]], [[1, 2, 8, 20, 21, 22, 27, 41, 59, 60, 65, 68, 71, 76, 78, 79, 84, 92], [5, 6, 7, 11, 13, 15, 29, 34, 36, 45, 51, 56, 69, 74, 75, 77, 80, 82, 85, 87, 88, 90], [0, 4, 9, 10, 14, 16, 17, 19, 23, 30, 31, 32, 33

In [16]:
for plan in nondominated_plans_old:
    print(scores(G, plan, G._ideal_population, obj_type))

[33, 0.08315652515430033]
[32, 0.7136165126980609]
[32, 0.7136165126980609]


In [11]:
start_time = time.perf_counter()
nondominated_plans_new = filter_dominated_plans_new(G, plans, obj_type)
print("Total time for filtering dominated plans =", round(time.perf_counter() - start_time, 2))

Total time for filtering dominated plans = 5.07


In [15]:
print(nondominated_plans_new)

[[[1, 2, 8, 20, 21, 22, 27, 41, 59, 60, 65, 68, 71, 76, 78, 79, 84, 92], [5, 6, 7, 11, 13, 15, 29, 34, 36, 45, 51, 56, 69, 74, 75, 77, 80, 82, 85, 87, 88, 90], [3, 12, 18, 24, 25, 26, 28, 38, 39, 44, 46, 47, 48, 54, 61, 67, 72, 83, 86, 94, 95, 96, 97, 98], [0, 4, 9, 10, 14, 16, 17, 19, 23, 30, 31, 32, 33, 35, 37, 40, 42, 43, 49, 50, 52, 53, 55, 57, 58, 62, 63, 64, 66, 70, 73, 81, 89, 91, 93]], [[1, 2, 8, 14, 20, 21, 22, 27, 41, 59, 60, 65, 68, 71, 76, 78, 79, 84, 92], [5, 6, 7, 11, 13, 15, 29, 34, 45, 51, 56, 69, 72, 75, 77, 80, 82, 85, 87, 88, 90], [3, 10, 12, 18, 24, 25, 26, 28, 38, 39, 44, 46, 47, 48, 54, 61, 67, 83, 86, 94, 95, 96, 97, 98], [0, 4, 9, 16, 17, 19, 23, 30, 31, 32, 33, 35, 36, 37, 40, 42, 43, 49, 50, 52, 53, 55, 57, 58, 62, 63, 64, 66, 70, 73, 74, 81, 89, 91, 93]]]


In [17]:
for plan in nondominated_plans_new:
    print(scores(G, plan, G._ideal_population, obj_type))

[32, 0.7136165126980609]
[33, 0.08315652515430033]
